In [1]:
import pandas as pd
import numpy as np
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import torch
from transformers import CLIPTokenizer, CLIPTextModel

# Download NLTK data files (only need to run once)
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

# Apply tqdm to all .apply() functions by using progress_apply
tqdm.pandas()

# Use GPU if available
"""
Reference: https://pytorch.org/get-started/locally/
"""
# Check for NVIDIA GPU
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use CUDA (NVIDIA GPU)
    print("Using NVIDIA GPU (CUDA)")
# Check for Mac Silicon GPU (MPS)
elif torch.backends.mps.is_available():
    device = torch.device("mps")  # Use Metal Performance Shaders (Mac Silicon GPU)
    print("Using Mac GPU (MPS)")
# Default to CPU if no GPU is available
else:
    device = torch.device("cpu")
    print("Using CPU")
# Reading the dataset
openmoji_df = pd.read_csv('../data/openmoji.csv')
llm_df = pd.read_parquet('../data/llmemoji.parquet')
# Convert Unicode string (e.g., 'U+1F600', 'U+263A,FE0F') to hex code ('1F600', '263A-FE0F').

def unicode_to_hex(unicode_str):
    unicode_str = unicode_str.replace(",", " ")  # Replace commas with spaces
    # First splitting the input string to a list of substrings
    # Loops though each substring
    # Removes the U+ prefix from each substring
    hex_values = [u.replace("U+", "") for u in unicode_str.split()]
    # Join the values with hyphens
    return "-".join(hex_values)

# Convert 'unicode' column in emojipedia_df and llm_df to 'hexcode'
llm_df['hexcode'] = llm_df['unicode'].progress_apply(unicode_to_hex)
# Making the hexcode uniform in all 3 dataframes for merging
# removing -f30f (differentiation between image type emoji and textual type emoji)
# removing -200d (differentiation for emoji with skin-tone)

openmoji_df['hexcode'] = openmoji_df['hexcode'].str.replace('-FE0F', '', regex=True)
openmoji_df['hexcode'] = openmoji_df['hexcode'].str.replace('-200D', '', regex=True)
llm_df['hexcode'] = llm_df['hexcode'].str.replace('-FE0F', '', regex=True)
llm_df['hexcode'] = llm_df['hexcode'].str.replace('-200D', '', regex=True)
openmoji_df['hexcode'] = openmoji_df['hexcode'].str.lower()
llm_df['hexcode'] = llm_df['hexcode'].str.lower()
# Selecting specific groups of emojis to train the model
openmoji_df = openmoji_df[openmoji_df["group"].isin(["smileys-emotion", "people-body"])]
openmoji_df = openmoji_df[~openmoji_df["subgroups"].isin(["person-symbol", "emotion"])]
# Checking for duplicates
duplicate_counts = openmoji_df['hexcode'].value_counts()
duplicates = duplicate_counts[duplicate_counts > 1]
print(duplicates)
# Checking for duplicates
duplicate_counts = llm_df['hexcode'].value_counts()
duplicates = duplicate_counts[duplicate_counts > 1]
print(duplicates)
# Removing duplicates
llm_df = llm_df[~llm_df.duplicated(subset=['hexcode'], keep=False)]
# Merge the dataframes on 'hexcode' with left join on openmoji_df
merged_df = openmoji_df.merge(llm_df, on='hexcode', how='left')
merged_df.info()
## Handling Tags and Descriptions
def clean_text(text):
    if not isinstance(text, str) or pd.isna(text) or text.strip().lower() == "nan":  
        return ""  # Return empty string for NaN or "nan" strings
    text = text.lower().strip() # Convert to lowercase and remove unnecessary spaces
    # Keep only letters, numbers, spaces, * and #
    text = re.sub(r'[^a-z0-9\s*#]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ', '.join(tokens)

def remove_duplicates(text):
    words = [word.strip() for word in text.split(",")]  # Split by commas and strip spaces
    unique_words = list(dict.fromkeys(words))
    return ', '.join(unique_words)  # Join back into a string
# Clean openmoji_annotation column
# Clean llm_tags
merged_df["cleaned_llm_tags"] = merged_df["tags_y"].progress_apply(clean_text)

# List of columns to merge (All tags)
columns_to_merge = ["tags_x", "cleaned_llm_tags"]

# Fill NaN with empty strings, then merge columns
merged_df["merged_tags"] = merged_df[columns_to_merge].fillna("").agg(
    lambda x: ", ".join(filter(None, map(str, x))), axis=1
)
# Remove duplicates
merged_df["final_tags"] = merged_df["merged_tags"].progress_apply(remove_duplicates)
# Function to handle concatenation with empty strings and NaN values
def merge_descriptions(row):
    parts = []
    if pd.notna(row['annotation']):
        annotation = row['annotation'].strip()
        if annotation:  # Ensure it's not empty after stripping
            parts.append(annotation)
    if pd.notna(row['LLM description']):
        description = row['LLM description'].strip()
        if description:
            parts.append(description)
    if pd.notna(row['final_tags']):
        tags = row['final_tags'].strip()
        if tags:
            parts.append(f"Tags: {tags}")  # Ensure the "Tags: " prefix is only added if valid

    return ". ".join(parts) if parts else None  # Return None if no valid parts

# Apply the function to each row
merged_df['merged_description'] = merged_df.apply(merge_descriptions, axis=1)
merged_df.info()
# Duplicate smiley emoji records so that the model has enough to learn from

# Filter rows with group "smileys-emotion"
smileys_df = merged_df[merged_df['group'] == 'smileys-emotion']

# Duplicate these rows 3 times
duplicated_smileys_df = pd.concat([smileys_df] * 2, ignore_index=True)

# Append back to the original DataFrame
merged_df = pd.concat([merged_df, duplicated_smileys_df], ignore_index=True)

# Shuffle the dataset to mix the records
merged_df = merged_df.sample(frac=1, random_state=42).reset_index(drop=True)
merged_df.info()
## Embedding Emoji Condition
"""
Reference: https://huggingface.co/docs/transformers/model_doc/clip
"""

# Load CLIP's tokenizer and text model.
clip_tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
clip_model = clip_model.to(device)
clip_model.eval()
"""
Reference: ChatGPT o3-mini-high
Prompt: Write a code to embed my description column in my df using CLIP-vit-base-patch32. Use Mean Pooling + L2 Normalization method to generate embeddings.

Reason: We're using Mean Pooling + L2 Normalization to retain fine-grained meanings related to gender, skin tone, emotions, and objects. We're also using L2 Normalization because they have a consistent scale, reducing variance in GAN training.
"""

def mean_pooling(model_output, attention_mask):
    """Mean pool the token embeddings."""
    token_embeddings = model_output.last_hidden_state  # (batch_size, sequence_length, hidden_dim)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, dim=1) / torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)

def embed_text(text):
    if pd.isna(text) or text.strip() == "":
        # Adjust the zero vector size to match CLIP's output dimension (512 for clip-vit-base-patch32)
        return np.zeros(512, dtype=np.float32)
    
    # Tokenize the input text
    inputs = clip_tokenizer(text, return_tensors="pt", truncation=True, max_length=77)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    
    # Disable gradients for inference
    with torch.no_grad():
        output = clip_model(**inputs)
    
    # Pool the token embeddings (mean pooling)
    pooled_embedding = mean_pooling(output, inputs["attention_mask"])
    
    # Optionally, you might want to L2 normalize the pooled embedding:
    pooled_embedding = torch.nn.functional.normalize(pooled_embedding, p=2, dim=-1)
    
    return pooled_embedding.squeeze().cpu().numpy().astype(np.float32)
# Apply CLIP embedding to your dataset
merged_df["combined_embedding"] = merged_df["merged_description"].progress_apply(embed_text)
merged_df.head()
## Linking Images
# Define base image path and brands
image_base_path = "../data/tensor_images/"
# brands = ["GoogleEmoji", "JoyPixelsEmoji", "TwitterEmoji"]
brands = ["OpenMojiEmoji"]

# Function to find all available image paths for a given hexcode
def get_image_paths(hexcode):
    image_paths = {}
    
    for brand in brands:
        brand_path = os.path.join(image_base_path, brand)
        if not os.path.exists(brand_path): # Skip if folder doesn't exist
            continue
            
        expected_filename = f"{hexcode}.pt"  # Adjust based on actual format
        
        if expected_filename in os.listdir(brand_path):
            image_paths[brand] = os.path.join(brand_path, expected_filename)

    return image_paths

# Expand dataframe with tqdm progress bar
expanded_rows = []
for _, row in tqdm(merged_df.iterrows(), total=len(merged_df), desc="Processing Hexcodes"):
    hexcode = row["hexcode"]
    embedding = row["combined_embedding"]

    image_paths = get_image_paths(hexcode)  # Get list of image paths
    
    if image_paths:  # If images exist, create multiple rows
        for brand, path in image_paths.items():
            expanded_rows.append({"hexcode": hexcode, "combined_embedding": embedding, "image_path": path})
    else:
        # If no images exist, optionally add a row with NaN for image_path
        expanded_rows.append({"hexcode": hexcode, "combined_embedding": embedding, "image_path": None})

# Convert to DataFrame
expanded_df = pd.DataFrame(expanded_rows)

# Optional: Drop rows where no image is found
expanded_df = expanded_df.dropna(subset=["image_path"]).reset_index(drop=True)
expanded_df.info()
output_file = '../data/processed_emoji_dataset.parquet'

# Check if the file exists and remove it
if os.path.exists(output_file):
    os.remove(output_file)

# Now save the DataFrame as Parquet
expanded_df[['combined_embedding', 'image_path']].to_parquet(output_file, index=False)


[nltk_data] Downloading package punkt_tab to C:\Users\Jyoti Prakash
[nltk_data]     Uprety\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Jyoti Prakash
[nltk_data]     Uprety\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Jyoti Prakash
[nltk_data]     Uprety\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Using NVIDIA GPU (CUDA)


100%|██████████| 5034/5034 [00:00<00:00, 983104.08it/s]


Series([], Name: count, dtype: int64)
hexcode
1f3cc-2640               4
1f575-2640               4
1f6b6-1f3fe-2642-27a1    4
1f3c3-2640-27a1          4
1f3c3-1f3ff-2640-27a1    4
                        ..
261d                     2
267e                     2
2139                     2
26f8                     2
1f470-1f3fb-2640         2
Name: count, Length: 1160, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2404 entries, 0 to 2403
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   emoji                  2404 non-null   object 
 1   hexcode                2404 non-null   object 
 2   group                  2404 non-null   object 
 3   subgroups              2404 non-null   object 
 4   annotation             2404 non-null   object 
 5   tags_x                 529 non-null    object 
 6   openmoji_tags          0 non-null      object 
 7   openmoji_author        2404 non-null 

100%|██████████| 2404/2404 [00:00<00:00, 456022.20it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2404 entries, 0 to 2403
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   emoji                  2404 non-null   object 
 1   hexcode                2404 non-null   object 
 2   group                  2404 non-null   object 
 3   subgroups              2404 non-null   object 
 4   annotation             2404 non-null   object 
 5   tags_x                 529 non-null    object 
 6   openmoji_tags          0 non-null      object 
 7   openmoji_author        2404 non-null   object 
 8   openmoji_date          2404 non-null   object 
 9   skintone               1875 non-null   object 
 10  skintone_combination   2198 non-null   object 
 11  skintone_base_emoji    2198 non-null   object 
 12  skintone_base_hexcode  2198 non-null   object 
 13  unicode_x              2404 non-null   object 
 14  order                  2404 non-null   float64
 15  char

Processing Hexcodes: 100%|██████████| 2712/2712 [00:09<00:00, 293.45it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2712 entries, 0 to 2711
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   hexcode             2712 non-null   object
 1   combined_embedding  2712 non-null   object
 2   image_path          2712 non-null   object
dtypes: object(3)
memory usage: 63.7+ KB
